<a href="https://colab.research.google.com/github/trucnguyen83/Injection-moulding/blob/master/Tsfresh_attempt_on_injection_moulding_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install tsfresh
!pip install tsfresh

     |████████████████████████████████| 92kB 2.4MB/s 
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=2abda9b4f1e4041ea7bb2afa31e22b1f554930ffb921c231f93d193a26fc4faf
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
Successfully built locket


In [ ]:
#prepare the environment
import pandas as pd
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#load the combine time series csv as dataframe
url = 'https://github.com/trucnguyen83/Injection-moulding/blob/master/combine_df.csv?raw=true'
combine_df = pd.read_csv(url)
combine_df.head()

,Unnamed: 0,Unnamed: 0.1,index,Timestamp,ns=2;i=201732,ns=2;i=201972,ns=2;i=202672,ns=2;i=202732,ns=2;i=140842,ns=2;i=148192,ns=2;i=148392,ns=2;i=148592,ns=2;i=148792,ns=2;i=148992,ns=2;i=149192,ns=2;i=192502,ns=2;i=201202,ns=2;i=201332,ns=2;i=201692,ns=2;i=202422,ns=2;i=202472,ns=2;i=202582,ns=2;i=207272,ns=2;i=207422,ns=2;i=207572,ns=2;i=207722,ns=2;i=207872,ns=2;i=227042,ns=2;i=227052,ns=2;i=115002,ns=2;i=194562,ns=2;i=199612,ns=2;i=231922,ns=2;i=34012,ns=2;i=34652,ns=2;i=596352,ns=2;i=209042,ns=2;i=202232,ns=2;i=21042,ns=2;i=21052,ns=2;i=21162,ns=2;i=21182,ns=2;i=21422,ns=2;i=21432,ns=2;i=21442,ns=2;i=21472,ns=2;i=21482,ns=2;i=22402,ns=2;i=766152,ns=2;i=767082,ns=2;i=767092,ns=2;i=201722,Time,ID,Type
0,0,0,0,2019/08/20 03:54:21.1466,6.754157,6.440265,0.000000,2.846,30.196,300.0,309.9,307.1,307.1,309.9,310.5,0,0.4,50,0.00000,0.0,0.000,0.194,279.7,295.1,310.0,310.0,314.9,1.291114,-229.97080,38,95.0,0.079369,175.579300,-0.34391,0.34391,-1.291114,42,0.0,5.989258,6.042665,11.34871,0.350963,-0.935,0,-0.039064,0.45,0,182.3557,120.8151,3.406,0.27,-1.243436,0.0,50,Bad2
1,1,1,1,2019/08/20 03:54:21.6483,6.754157,6.440265,0.000000,2.846,30.196,300.0,309.9,307.2,307.1,309.9,310.5,0,0.4,50,0.00000,0.0,0.000,0.194,279.7,295.1,310.0,310.0,314.9,1.475559,-84.92697,38,95.0,0.079369,84.926970,-0.34391,0.34391,-1.291114,42,0.0,5.989258,6.103702,11.34871,0.427259,-1.045,0,-0.029298,0.45,0,182.4778,120.8151,3.406,0.27,-1.243436,0.5,50,Bad2
2,2,2,2,2019/08/20 03:54:22.1420,6.754157,6.440265,0.000000,2.846,30.196,300.0,309.9,307.2,307.1,310.0,310.5,0,0.4,50,0.00000,0.0,0.000,0.194,279.7,295.1,310.0,310.0,314.9,1.475559,-17.17624,38,95.0,0.079369,9.542356,0.34391,0.34391,-1.475559,42,0.0,5.989258,5.958739,11.34871,0.427259,-0.935,0,-0.024415,0.45,0,182.3252,120.8151,3.406,0.27,-1.243436,1.0,50,Bad2
3,3,3,3,2019/08/20 03:54:22.6506,6.754157,6.440265,0.000000,2.846,30.196,300.0,309.9,307.1,307.2,310.0,310.5,0,0.4,50,0.00000,0.0,0.000,0.194,279.7,295.2,310.0,310.0,314.9,-501.505700,0.00000,38,95.0,0.079369,0.000000,-0.34391,0.34391,501.690200,42,0.0,5.989258,5.951110,35.00039,0.396741,-1.100,0,-0.043947,0.36,0,182.3252,120.8151,3.406,0.27,-0.155429,1.5,50,Bad2
4,4,4,4,2019/08/20 03:54:23.1593,6.754157,6.440265,0.950157,2.846,30.196,300.0,309.9,307.1,307.2,309.9,310.5,0,0.4,50,20.37149,0.0,1745.007,0.194,279.7,295.2,310.0,310.0,314.9,-500.767900,0.00000,38,95.0,-0.039684,0.000000,-0.34391,0.34391,500.767900,42,0.0,5.989258,6.080813,35.00039,80.416280,63.690,0,50.348220,0.45,0,181.8980,120.8151,3.406,0.27,1602.633000,2.0,50,Bad2


In [ ]:
#prepare the labels in another df
combresult = pd.DataFrame()
combresult[['ID','Type']] = combine_df[['ID','Type']]
combresult.head()

,ID,Type
0,50,Bad2
1,50,Bad2
2,50,Bad2
3,50,Bad2
4,50,Bad2


In [ ]:
#prepare the time series df to feed tsfresh and removing the label
combine_feed_df = combine_df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1','index','Timestamp','Type'])
combine_feed_df.head()

,ns=2;i=201732,ns=2;i=201972,ns=2;i=202672,ns=2;i=202732,ns=2;i=140842,ns=2;i=148192,ns=2;i=148392,ns=2;i=148592,ns=2;i=148792,ns=2;i=148992,ns=2;i=149192,ns=2;i=192502,ns=2;i=201202,ns=2;i=201332,ns=2;i=201692,ns=2;i=202422,ns=2;i=202472,ns=2;i=202582,ns=2;i=207272,ns=2;i=207422,ns=2;i=207572,ns=2;i=207722,ns=2;i=207872,ns=2;i=227042,ns=2;i=227052,ns=2;i=115002,ns=2;i=194562,ns=2;i=199612,ns=2;i=231922,ns=2;i=34012,ns=2;i=34652,ns=2;i=596352,ns=2;i=209042,ns=2;i=202232,ns=2;i=21042,ns=2;i=21052,ns=2;i=21162,ns=2;i=21182,ns=2;i=21422,ns=2;i=21432,ns=2;i=21442,ns=2;i=21472,ns=2;i=21482,ns=2;i=22402,ns=2;i=766152,ns=2;i=767082,ns=2;i=767092,ns=2;i=201722,Time,ID
0,6.754157,6.440265,0.000000,2.846,30.196,300.0,309.9,307.1,307.1,309.9,310.5,0,0.4,50,0.00000,0.0,0.000,0.194,279.7,295.1,310.0,310.0,314.9,1.291114,-229.97080,38,95.0,0.079369,175.579300,-0.34391,0.34391,-1.291114,42,0.0,5.989258,6.042665,11.34871,0.350963,-0.935,0,-0.039064,0.45,0,182.3557,120.8151,3.406,0.27,-1.243436,0.0,50
1,6.754157,6.440265,0.000000,2.846,30.196,300.0,309.9,307.2,307.1,309.9,310.5,0,0.4,50,0.00000,0.0,0.000,0.194,279.7,295.1,310.0,310.0,314.9,1.475559,-84.92697,38,95.0,0.079369,84.926970,-0.34391,0.34391,-1.291114,42,0.0,5.989258,6.103702,11.34871,0.427259,-1.045,0,-0.029298,0.45,0,182.4778,120.8151,3.406,0.27,-1.243436,0.5,50
2,6.754157,6.440265,0.000000,2.846,30.196,300.0,309.9,307.2,307.1,310.0,310.5,0,0.4,50,0.00000,0.0,0.000,0.194,279.7,295.1,310.0,310.0,314.9,1.475559,-17.17624,38,95.0,0.079369,9.542356,0.34391,0.34391,-1.475559,42,0.0,5.989258,5.958739,11.34871,0.427259,-0.935,0,-0.024415,0.45,0,182.3252,120.8151,3.406,0.27,-1.243436,1.0,50
3,6.754157,6.440265,0.000000,2.846,30.196,300.0,309.9,307.1,307.2,310.0,310.5,0,0.4,50,0.00000,0.0,0.000,0.194,279.7,295.2,310.0,310.0,314.9,-501.505700,0.00000,38,95.0,0.079369,0.000000,-0.34391,0.34391,501.690200,42,0.0,5.989258,5.951110,35.00039,0.396741,-1.100,0,-0.043947,0.36,0,182.3252,120.8151,3.406,0.27,-0.155429,1.5,50
4,6.754157,6.440265,0.950157,2.846,30.196,300.0,309.9,307.1,307.2,309.9,310.5,0,0.4,50,20.37149,0.0,1745.007,0.194,279.7,295.2,310.0,310.0,314.9,-500.767900,0.00000,38,95.0,-0.039684,0.000000,-0.34391,0.34391,500.767900,42,0.0,5.989258,6.080813,35.00039,80.416280,63.690,0,50.348220,0.45,0,181.8980,120.8151,3.406,0.27,1602.633000,2.0,50


In [ ]:
#check the types of all the feed df columns is float
combine_feed_df.dtypes

ns=2;i=201732    float64
ns=2;i=201972    float64
ns=2;i=202672    float64
ns=2;i=202732    float64
ns=2;i=140842    float64
ns=2;i=148192    float64
ns=2;i=148392    float64
ns=2;i=148592    float64
ns=2;i=148792    float64
ns=2;i=148992    float64
ns=2;i=149192    float64
ns=2;i=192502      int64
ns=2;i=201202    float64
ns=2;i=201332      int64
ns=2;i=201692    float64
ns=2;i=202422    float64
ns=2;i=202472    float64
ns=2;i=202582    float64
ns=2;i=207272    float64
ns=2;i=207422    float64
ns=2;i=207572    float64
ns=2;i=207722    float64
ns=2;i=207872    float64
ns=2;i=227042    float64
ns=2;i=227052    float64
ns=2;i=115002      int64
ns=2;i=194562    float64
ns=2;i=199612    float64
ns=2;i=231922    float64
ns=2;i=34012     float64
ns=2;i=34652     float64
ns=2;i=596352    float64
ns=2;i=209042      int64
ns=2;i=202232    float64
ns=2;i=21042     float64
ns=2;i=21052     float64
ns=2;i=21162     float64
ns=2;i=21182     float64
ns=2;i=21422     float64
ns=2;i=21432       int64


In [ ]:
#convert all columns of feed df to float
combine_feed_df = combine_feed_df.astype(float)
combine_feed_df.dtypes

ns=2;i=201732    float64
ns=2;i=201972    float64
ns=2;i=202672    float64
ns=2;i=202732    float64
ns=2;i=140842    float64
ns=2;i=148192    float64
ns=2;i=148392    float64
ns=2;i=148592    float64
ns=2;i=148792    float64
ns=2;i=148992    float64
ns=2;i=149192    float64
ns=2;i=192502    float64
ns=2;i=201202    float64
ns=2;i=201332    float64
ns=2;i=201692    float64
ns=2;i=202422    float64
ns=2;i=202472    float64
ns=2;i=202582    float64
ns=2;i=207272    float64
ns=2;i=207422    float64
ns=2;i=207572    float64
ns=2;i=207722    float64
ns=2;i=207872    float64
ns=2;i=227042    float64
ns=2;i=227052    float64
ns=2;i=115002    float64
ns=2;i=194562    float64
ns=2;i=199612    float64
ns=2;i=231922    float64
ns=2;i=34012     float64
ns=2;i=34652     float64
ns=2;i=596352    float64
ns=2;i=209042    float64
ns=2;i=202232    float64
ns=2;i=21042     float64
ns=2;i=21052     float64
ns=2;i=21162     float64
ns=2;i=21182     float64
ns=2;i=21422     float64
ns=2;i=21432     float64


In [ ]:
#scale the feed data, except for the ID and Time columns
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(combine_feed_df.iloc[:,0:-2]) 
combine_feed_df.iloc[:,0:-2] = scaled_values
combine_feed_df.head()

,ns=2;i=201732,ns=2;i=201972,ns=2;i=202672,ns=2;i=202732,ns=2;i=140842,ns=2;i=148192,ns=2;i=148392,ns=2;i=148592,ns=2;i=148792,ns=2;i=148992,ns=2;i=149192,ns=2;i=192502,ns=2;i=201202,ns=2;i=201332,ns=2;i=201692,ns=2;i=202422,ns=2;i=202472,ns=2;i=202582,ns=2;i=207272,ns=2;i=207422,ns=2;i=207572,ns=2;i=207722,ns=2;i=207872,ns=2;i=227042,ns=2;i=227052,ns=2;i=115002,ns=2;i=194562,ns=2;i=199612,ns=2;i=231922,ns=2;i=34012,ns=2;i=34652,ns=2;i=596352,ns=2;i=209042,ns=2;i=202232,ns=2;i=21042,ns=2;i=21052,ns=2;i=21162,ns=2;i=21182,ns=2;i=21422,ns=2;i=21432,ns=2;i=21442,ns=2;i=21472,ns=2;i=21482,ns=2;i=22402,ns=2;i=766152,ns=2;i=767082,ns=2;i=767092,ns=2;i=201722,Time,ID
0,1.0,0.0,0.000000,0.010363,0.727637,0.555556,0.333333,0.555556,0.625,0.0,0.923077,0.0,0.0,0.0,0.405207,0.0,0.00000,0.0,0.529412,0.714286,0.5,0.5,0.25,0.994662,0.052798,0.0,0.5,0.730769,0.886076,0.630435,0.391217,0.005338,1.0,0.023904,0.049981,0.237249,0.324111,0.007133,0.505800,0.58556,0.022014,0.714286,0.033531,0.109034,0.0,0.191499,0.199704,0.021043,0.0,50.0
1,1.0,0.0,0.000000,0.010363,0.727637,0.555556,0.333333,0.666667,0.625,0.0,0.923077,0.0,0.0,0.0,0.405207,0.0,0.00000,0.0,0.529412,0.714286,0.5,0.5,0.25,0.995018,0.213305,0.0,0.5,0.730769,0.785865,0.630435,0.391217,0.005338,1.0,0.023904,0.049981,0.239310,0.324111,0.008035,0.504971,0.58556,0.022186,0.714286,0.033531,0.110642,0.0,0.191499,0.199704,0.021043,0.5,50.0
2,1.0,0.0,0.000000,0.010363,0.727637,0.555556,0.333333,0.666667,0.625,0.5,0.923077,0.0,0.0,0.0,0.405207,0.0,0.00000,0.0,0.529412,0.714286,0.5,0.5,0.25,0.995018,0.288279,0.0,0.5,0.730769,0.702532,0.634387,0.391217,0.004982,1.0,0.023904,0.049981,0.234415,0.324111,0.008035,0.505800,0.58556,0.022272,0.714286,0.033531,0.108632,0.0,0.191499,0.199704,0.021043,1.0,50.0
3,1.0,0.0,0.000000,0.010363,0.727637,0.555556,0.333333,0.555556,0.750,0.5,0.923077,0.0,0.0,0.0,0.405207,0.0,0.00000,0.0,0.529412,0.857143,0.5,0.5,0.25,0.024555,0.307286,0.0,0.5,0.730769,0.691983,0.630435,0.391217,0.975801,1.0,0.023904,0.049981,0.234158,1.000000,0.007674,0.504557,0.58556,0.021928,0.710843,0.033531,0.108632,0.0,0.191499,0.199704,0.021649,1.5,50.0
4,1.0,0.0,0.140677,0.010363,0.727637,0.555556,0.333333,0.555556,0.750,0.0,0.923077,0.0,0.0,0.0,0.791712,0.0,0.96485,0.0,0.529412,0.857143,0.5,0.5,0.25,0.025979,0.307286,0.0,0.5,0.153846,0.691983,0.630435,0.391217,0.974021,1.0,0.023904,0.049981,0.238537,1.000000,0.954587,0.992543,0.58556,0.909365,0.714286,0.033531,0.103005,0.0,0.191499,0.199704,0.914617,2.0,50.0


In [ ]:
#feed tsfresh to get the X before training classification model
extraction_settings = ComprehensiveFCParameters()

X = extract_features(combine_feed_df, column_id='ID', column_sort='Time',
                     default_fc_parameters=extraction_settings,
                     impute_function=impute)
X.head()

Feature Extraction: 100%|██████████| 5/5 [08:50<00:00, 106.13s/it]


variable,ns=2;i=115002__abs_energy,ns=2;i=115002__absolute_sum_of_changes,"ns=2;i=115002__agg_autocorrelation__f_agg_""mean""__maxlag_40","ns=2;i=115002__agg_autocorrelation__f_agg_""median""__maxlag_40","ns=2;i=115002__agg_autocorrelation__f_agg_""var""__maxlag_40","ns=2;i=115002__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""","ns=2;i=115002__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","ns=2;i=115002__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","ns=2;i=115002__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","ns=2;i=115002__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""","ns=2;i=115002__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""mean""","ns=2;i=115002__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""min""","ns=2;i=115002__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""var""","ns=2;i=115002__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","ns=2;i=115002__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","ns=2;i=115002__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","ns=2;i=115002__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""var""","ns=2;i=115002__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""max""","ns=2;i=115002__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""mean""","ns=2;i=115002__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""min""","ns=2;i=115002__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""var""","ns=2;i=115002__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""max""","ns=2;i=115002__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""mean""","ns=2;i=115002__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""min""","ns=2;i=115002__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""var""","ns=2;i=115002__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""max""","ns=2;i=115002__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""mean""","ns=2;i=115002__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""min""","ns=2;i=115002__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""var""","ns=2;i=115002__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""max""","ns=2;i=115002__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""mean""","ns=2;i=115002__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""min""","ns=2;i=115002__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""var""","ns=2;i=115002__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""max""","ns=2;i=115002__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""mean""","ns=2;i=115002__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""min""","ns=2;i=115002__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""var""","ns=2;i=115002__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","ns=2;i=115002__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""mean""","ns=2;i=115002__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""min""",...,ns=2;i=767092__ratio_beyond_r_sigma__r_7,ns=2;i=767092__ratio_value_number_to_time_series_length,ns=2;i=767092__sample_entropy,ns=2;i=767092__skewness,ns=2;i=767092__spkt_welch_density__coeff_2,ns=2;i=767092__spkt_welch_density__coeff_5,ns=2;i=767092__spkt_welch_density__coeff_8,ns=2;i=767092__standard_deviation,ns=2;i=767092__sum_of_reoccurring_data_points,ns=2;i=767092__sum_of_reoccurring_values,ns=2;i=767092__sum_values,ns=2;i=767092__symmetry_looking__r_0.0,ns=2;i=767092__symmetry_looking__r_0.05,ns=2;i=767092__symmetry_looking__r_0.1,ns=2;i=767092__symmetry_looking__r_0.15000000000000002,ns=2;i=767092__symmetry_looking__r_0.2,ns=2;i=767092__symmetry_looking__r_0.25,ns=2;i=767092__symmetry_looking__r_0.30000000000000004,ns=2;i=767092__symmetry_looking__r_0.35000000000000003,ns=2;i=767092__symmetry_looking__r_0.4,ns=2;i=767092__symmetry_looking__r_0.45,ns=2;i=767092__symmetry_looking__r_0.5

In [ ]:
#remove duplications in the label df (only 203 rows) so that it can match to X
combresult.drop_duplicates(subset ="ID",keep='first',inplace=True) 
combresult.head()

,ID,Type
0,50,Bad2
62,119,Good
123,8,Bad1
184,186,Good
245,44,Bad2


In [ ]:
#sort the index by ID that y can be matched to X
combresult.sort_values(by=['ID'], inplace = True)
combresult.head()

,ID,Type
3136,1,Bad1
1609,2,Bad1
5068,3,Bad1
11828,4,Bad1
8017,5,Bad1


In [ ]:
#reset the index by ID that y can be matched to X
combresult = combresult.reset_index().set_index('ID')
combresult.head()

,index,Type
ID,,
1,3136,Bad1
2,1609,Bad1
3,5068,Bad1
4,11828,Bad1
5,8017,Bad1


In [ ]:
#define y, y must be a series, not df
y = combresult['Type']

In [ ]:
#select only important or relevant features
#read the algorithm of how to select relevent features here, https://arxiv.org/pdf/1610.07717.pdf, applicable only for tsfresh
X_filtered = select_features(X, y)
X_filtered.head()

variable,"ns=2;i=115002__fft_coefficient__attr_""abs""__coeff_4","ns=2;i=115002__fft_coefficient__attr_""abs""__coeff_13","ns=2;i=148592__linear_trend__attr_""slope""","ns=2;i=209042__fft_coefficient__attr_""angle""__coeff_30","ns=2;i=148592__linear_trend__attr_""intercept""","ns=2;i=209042__fft_coefficient__attr_""abs""__coeff_2","ns=2;i=140842__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""mean""","ns=2;i=209042__fft_coefficient__attr_""real""__coeff_30","ns=2;i=767092__fft_coefficient__attr_""angle""__coeff_16","ns=2;i=209042__fft_coefficient__attr_""angle""__coeff_28","ns=2;i=209042__fft_coefficient__attr_""abs""__coeff_15",ns=2;i=140842__spkt_welch_density__coeff_8,ns=2;i=140842__variation_coefficient,ns=2;i=148592__time_reversal_asymmetry_statistic__lag_3,ns=2;i=140842__variance,ns=2;i=140842__standard_deviation,"ns=2;i=209042__fft_coefficient__attr_""abs""__coeff_19","ns=2;i=140842__fft_coefficient__attr_""abs""__coeff_9","ns=2;i=148592__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","ns=2;i=148592__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""mean""","ns=2;i=209042__fft_coefficient__attr_""abs""__coeff_23",ns=2;i=140842__spkt_welch_density__coeff_5,"ns=2;i=209042__fft_coefficient__attr_""real""__coeff_28","ns=2;i=21442__fft_coefficient__attr_""angle""__coeff_6","ns=2;i=140842__linear_trend__attr_""stderr""","ns=2;i=209042__fft_coefficient__attr_""abs""__coeff_30","ns=2;i=209042__fft_coefficient__attr_""abs""__coeff_16",ns=2;i=140842__spkt_welch_density__coeff_2,"ns=2;i=209042__fft_coefficient__attr_""real""__coeff_14","ns=2;i=115002__fft_coefficient__attr_""abs""__coeff_7","ns=2;i=140842__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""max""","ns=2;i=148592__cwt_coefficients__coeff_8__w_20__widths_(2, 5, 10, 20)","ns=2;i=115002__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""mean""","ns=2;i=115002__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""mean""","ns=2;i=148592__cwt_coefficients__coeff_6__w_20__widths_(2, 5, 10, 20)","ns=2;i=140842__fft_coefficient__attr_""abs""__coeff_12","ns=2;i=140842__fft_coefficient__attr_""abs""__coeff_16","ns=2;i=148592__cwt_coefficients__coeff_9__w_20__widths_(2, 5, 10, 20)","ns=2;i=148592__cwt_coefficients__coeff_7__w_20__widths_(2, 5, 10, 20)","ns=2;i=115002__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""mean""",...,ns=2;i=202422__number_cwt_peaks__n_5,ns=2;i=202422__large_standard_deviation__r_0.45,ns=2;i=202422__large_standard_deviation__r_0.4,ns=2;i=202472__symmetry_looking__r_0.2,ns=2;i=202472__symmetry_looking__r_0.15000000000000002,ns=2;i=202472__large_standard_deviation__r_0.45,ns=2;i=202472__large_standard_deviation__r_0.4,ns=2;i=202472__large_standard_deviation__r_0.35000000000000003,ns=2;i=202732__large_standard_deviation__r_0.2,ns=2;i=202232__symmetry_looking__r_0.1,ns=2;i=202732__large_standard_deviation__r_0.35000000000000003,ns=2;i=202732__large_standard_deviation__r_0.30000000000000004,ns=2;i=202732__large_standard_deviation__r_0.25,ns=2;i=202732__symmetry_looking__r_0.1,ns=2;i=201202__ratio_beyond_r_sigma__r_0.5,"ns=2;i=140842__augmented_dickey_fuller__attr_""pvalue""__autolag_""AIC""",ns=2;i=201202__large_standard_deviation__r_0.05,ns=2;i=201202__large_standard_deviation__r_0.1,ns=2;i=201202__large_standard_deviation__r_0.2,ns=2;i=201202__large_standard_deviation__r_0.9500000000000001,ns=2;i=201202__large_standard_deviation__r_0.9,ns=2;i=201202__large_standard_deviation__r_0.8500000000000001,ns=2;i=201202__large_standard_deviation__r_0.8,ns=2;i=201202__large_standard_deviation__r_0.75,ns=2;i=201202__large_standard_deviation__r_0.7000000000000001,ns=2;i=201202__large_standard_deviation__r_0.65,ns=2;i=201202__large_standard_deviation__r_0.6000000000000001,ns=2;i=201202__large_standard_deviation__r_0.55,ns=2;i=201202__large_standard_deviation__r_0.5,ns=2;i=201202__large_standard_deviation__r_0.45,ns=2;i=201202__large_standard_deviation__r_0.4,ns=2;i=201202__large_standard_deviation__r_0.350000

In [ ]:
#define the train, test set with full features and selected features
X_full_train, X_full_test, y_train, y_test = train_test_split(X, y, random_state = 4, test_size=.4)
X_filtered_train, X_filtered_test = X_full_train[X_filtered.columns], X_full_test[X_filtered.columns]

In [ ]:
#train the full features with Decision Tree model and check the f1 score
classifier_full = DecisionTreeClassifier()
classifier_full.fit(X_full_train, y_train)
print(classification_report(y_test, classifier_full.predict(X_full_test)))

              precision    recall  f1-score   support

        Bad1       0.67      0.50      0.57         4
        Bad2       1.00      1.00      1.00        18
        Bad3       1.00      1.00      1.00         3
        Bad4       1.00      0.75      0.86         4
        Bad5       1.00      1.00      1.00        13
        Good       0.95      1.00      0.98        40

    accuracy                           0.96        82
   macro avg       0.94      0.88      0.90        82
weighted avg       0.96      0.96      0.96        82



In [ ]:
#train the selected features with Decision Tree model and check the f1 score
classifier_filtered = DecisionTreeClassifier()
classifier_filtered.fit(X_filtered_train, y_train)
print(classification_report(y_test, classifier_filtered.predict(X_filtered_test)))

              precision    recall  f1-score   support

        Bad1       0.60      0.75      0.67         4
        Bad2       1.00      1.00      1.00        18
        Bad3       1.00      1.00      1.00         3
        Bad4       0.67      0.50      0.57         4
        Bad5       1.00      0.92      0.96        13
        Good       0.98      1.00      0.99        40

    accuracy                           0.95        82
   macro avg       0.87      0.86      0.86        82
weighted avg       0.95      0.95      0.95        82



In [ ]:
#try to train full features with Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier_full = RandomForestClassifier(n_estimators=10, random_state=1, oob_score=True)
classifier_full.fit(X_full_train, y_train)
print(classification_report(y_test, classifier_full.predict(X_full_test)))

              precision    recall  f1-score   support

        Bad1       0.67      0.50      0.57         4
        Bad2       1.00      1.00      1.00        18
        Bad3       0.60      1.00      0.75         3
        Bad4       0.00      0.00      0.00         4
        Bad5       0.86      0.92      0.89        13
        Good       0.95      1.00      0.98        40

    accuracy                           0.91        82
   macro avg       0.68      0.74      0.70        82
weighted avg       0.87      0.91      0.89        82



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#try to train selected features with Random Forest
classifier_filtered = RandomForestClassifier(n_estimators=10, random_state=1, oob_score=True)
classifier_filtered.fit(X_filtered_train, y_train)
print(classification_report(y_test, classifier_filtered.predict(X_filtered_test)))

              precision    recall  f1-score   support

        Bad1       0.60      0.75      0.67         4
        Bad2       1.00      1.00      1.00        18
        Bad3       0.75      1.00      0.86         3
        Bad4       0.00      0.00      0.00         4
        Bad5       0.92      0.92      0.92        13
        Good       0.98      1.00      0.99        40

    accuracy                           0.93        82
   macro avg       0.71      0.78      0.74        82
weighted avg       0.90      0.93      0.91        82



In [ ]:
print("Full obb_score: ", classifier_full.oob_score_)
print("Filter obb_score: ", classifier_filtered.oob_score_) 

Full obb_score:  0.8677685950413223
Filter obb_score:  0.9173553719008265


In [ ]:
#try to train full features with Gradient Boosting 
from sklearn.ensemble import GradientBoostingClassifier
classifier_full = GradientBoostingClassifier (n_estimators=10, random_state=4)
classifier_full.fit(X_full_train, y_train)
print(classification_report(y_test, classifier_full.predict(X_full_test)))

              precision    recall  f1-score   support

        Bad1       0.00      0.00      0.00         4
        Bad2       1.00      1.00      1.00        18
        Bad3       1.00      1.00      1.00         3
        Bad4       1.00      1.00      1.00         4
        Bad5       1.00      0.92      0.96        13
        Good       0.89      1.00      0.94        40

    accuracy                           0.94        82
   macro avg       0.81      0.82      0.82        82
weighted avg       0.90      0.94      0.92        82



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#try to train selected features with Gradient Boosting 
classifier_filtered = GradientBoostingClassifier (n_estimators=10, random_state=4)
classifier_filtered.fit(X_filtered_train, y_train)
print(classification_report(y_test, classifier_filtered.predict(X_filtered_test)))

              precision    recall  f1-score   support

        Bad1       0.00      0.00      0.00         4
        Bad2       1.00      1.00      1.00        18
        Bad3       1.00      1.00      1.00         3
        Bad4       1.00      0.75      0.86         4
        Bad5       1.00      0.92      0.96        13
        Good       0.89      1.00      0.94        40

    accuracy                           0.93        82
   macro avg       0.81      0.78      0.79        82
weighted avg       0.90      0.93      0.91        82

